# Исследовательский анализ данных: Предсказание выпуска студентов

## Цель исследования
Провести детальный анализ данных об академических успехах студентов для выявления закономерностей, влияющих на вероятность выпуска. Исследование направлено на понимание факторов успеха и построение качественной модели предсказания.

## Структура исследования
1. Загрузка и первичный анализ данных
2. Анализ целевой переменной
3. Исследование признаков и их распределений
4. Корреляционный анализ
5. Тестирование гипотез
6. Выводы и рекомендации


In [ ]:
# Импорт библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Настройка визуализации
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Настройка pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Библиотеки успешно импортированы!")
print(f"Pandas версия: {pd.__version__}")
print(f"NumPy версия: {np.__version__}")


## 1. Загрузка и первичный анализ данных


In [ ]:
# Загрузка данных
DATA_PATH = Path("data/raw:")

# Загрузка основных данных
try:
    data_df = pd.read_csv(DATA_PATH / "data.csv")
    marking_df = pd.read_csv(DATA_PATH / "marking.csv")
    sample_submission_df = pd.read_csv(DATA_PATH / "sample_submission.csv")
    print("✓ Данные успешно загружены")
except FileNotFoundError:
    # Альтернативный путь
    DATA_PATH = Path("data/raw:")
    data_df = pd.read_csv(DATA_PATH / "data.csv")
    marking_df = pd.read_csv(DATA_PATH / "marking.csv")
    sample_submission_df = pd.read_csv(DATA_PATH / "sample_submission.csv")
    print("✓ Данные успешно загружены (альтернативный путь)")

print(f"\n📊 data.csv: {len(data_df):,} записей, {len(data_df.columns)} колонок")
print(f"📊 marking.csv: {len(marking_df):,} записей, {len(marking_df.columns)} колонок")
print(f"📊 sample_submission.csv: {len(sample_submission_df):,} записей")


In [ ]:
# Первичный осмотр данных
print("=" * 60)
print("СТРУКТУРА ДАННЫХ")
print("=" * 60)

print("\n1. data.csv (Оценки студентов):")
print(data_df.head())
print(f"\nКолонки: {list(data_df.columns)}")
print(f"\nТипы данных:")
print(data_df.dtypes)

print("\n2. marking.csv (Информация о студентах):")
print(marking_df.head())
print(f"\nКолонки: {list(marking_df.columns)}")
print(f"\nТипы данных:")
print(marking_df.dtypes)


In [ ]:
# Базовая статистика по данным
print("=" * 60)
print("БАЗОВАЯ СТАТИСТИКА")
print("=" * 60)

print("\n📚 data.csv:")
print(f"  • Уникальных студентов: {data_df['PK'].nunique():,}")
print(f"  • Уникальных семестров: {sorted(data_df['SEMESTER'].unique())}")
print(f"  • Уникальных дисциплин: {data_df['DNAME'].nunique():,}")
print(f"  • Типы оценок: {data_df['TYPE'].value_counts().to_dict()}")

print("\n👥 marking.csv:")
print(f"  • Уникальных студентов: {marking_df['ИД'].nunique():,}")
if 'выпуск' in marking_df.columns:
    print(f"  • Распределение целевой переменной:")
    print(marking_df['выпуск'].value_counts())

# Проверка пересечения студентов
students_in_data = set(data_df['PK'].unique())
students_in_marking = set(marking_df['ИД'].unique())
common_students = students_in_data & students_in_marking
print(f"\n🔗 Пересечение:")
print(f"  • Студентов в data.csv: {len(students_in_data):,}")
print(f"  • Студентов в marking.csv: {len(students_in_marking):,}")
print(f"  • Общих студентов: {len(common_students):,} ({len(common_students)/len(students_in_data)*100:.1f}%)")


In [ ]:
# Анализ пропущенных значений
print("=" * 60)
print("АНАЛИЗ ПРОПУЩЕННЫХ ЗНАЧЕНИЙ")
print("=" * 60)

print("\n📊 data.csv:")
missing_data = data_df.isnull().sum()
missing_data_pct = (missing_data / len(data_df)) * 100
missing_df = pd.DataFrame({
    'Количество': missing_data,
    'Процент': missing_data_pct
}).sort_values('Количество', ascending=False)
print(missing_df[missing_df['Количество'] > 0])

print("\n📊 marking.csv:")
missing_marking = marking_df.isnull().sum()
missing_marking_pct = (missing_marking / len(marking_df)) * 100
missing_marking_df = pd.DataFrame({
    'Количество': missing_marking,
    'Процент': missing_marking_pct
}).sort_values('Количество', ascending=False)
print(missing_marking_df[missing_marking_df['Количество'] > 0])


## 2. Анализ целевой переменной


In [ ]:
# Анализ целевой переменной
if 'выпуск' in marking_df.columns:
    target_counts = marking_df['выпуск'].value_counts()
    target_pct = (target_counts / len(marking_df)) * 100
    
    print("=" * 60)
    print("АНАЛИЗ ЦЕЛЕВОЙ ПЕРЕМЕННОЙ 'выпуск'")
    print("=" * 60)
    print("\nРаспределение:")
    for val, count in target_counts.items():
        print(f"  {val}: {count:,} ({target_pct[val]:.2f}%)")
    
    # Визуализация
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Столбчатая диаграмма
    axes[0].bar(target_counts.index, target_counts.values, color=['#2ecc71', '#e74c3c'])
    axes[0].set_title('Распределение целевой переменной', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Выпуск')
    axes[0].set_ylabel('Количество студентов')
    axes[0].grid(axis='y', alpha=0.3)
    
    # Круговая диаграмма
    axes[1].pie(target_counts.values, labels=target_counts.index, autopct='%1.1f%%',
                colors=['#2ecc71', '#e74c3c'], startangle=90)
    axes[1].set_title('Доля студентов по статусу выпуска', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Сохранение для дальнейшего использования
    marking_processed = marking_df.copy()
    marking_processed['target'] = (marking_processed['выпуск'] == 'выпустился').astype(int)
    print(f"\n✓ Целевая переменная закодирована: 1 = выпустился, 0 = не выпустился")
else:
    print("Целевая переменная 'выпуск' не найдена в данных")


## 3. Исследование признаков и их распределений


In [ ]:
# Подготовка данных для анализа
# Объединение данных для полного анализа
data_merged = data_df.merge(
    marking_df.rename(columns={'ИД': 'PK'})[['PK', 'выпуск', 'Факультет', 'Направление', 'год поступления']],
    on='PK',
    how='left'
)

# Создание целевой переменной
if 'выпуск' in data_merged.columns:
    data_merged['target'] = (data_merged['выпуск'] == 'выпустился').astype(int)

print("✓ Данные объединены для анализа")
print(f"  Всего записей: {len(data_merged):,}")
print(f"  Уникальных студентов: {data_merged['PK'].nunique():,}")


In [ ]:
# Анализ распределения по семестрам
print("=" * 60)
print("АНАЛИЗ РАСПРЕДЕЛЕНИЯ ПО СЕМЕСТРАМ")
print("=" * 60)

semester_dist = data_merged['SEMESTER'].value_counts().sort_index()
print("\nРаспределение записей по семестрам:")
for sem, count in semester_dist.items():
    pct = count / len(data_merged) * 100
    print(f"  Семестр {sem}: {count:,} записей ({pct:.1f}%)")

# Визуализация
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Столбчатая диаграмма
axes[0].bar(semester_dist.index, semester_dist.values, color='steelblue', alpha=0.7)
axes[0].set_title('Распределение записей по семестрам', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Номер семестра')
axes[0].set_ylabel('Количество записей')
axes[0].grid(axis='y', alpha=0.3)

# Семестр vs Целевая переменная (если доступна)
if 'target' in data_merged.columns:
    semester_target = data_merged.groupby(['SEMESTER', 'target']).size().unstack(fill_value=0)
    semester_target_pct = semester_target.div(semester_target.sum(axis=1), axis=0) * 100
    
    x = np.arange(len(semester_target_pct.index))
    width = 0.35
    
    axes[1].bar(x - width/2, semester_target_pct[0], width, label='Не выпустился', alpha=0.7)
    axes[1].bar(x + width/2, semester_target_pct[1], width, label='Выпустился', alpha=0.7)
    axes[1].set_title('Доля выпустившихся по семестрам', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Номер семестра')
    axes[1].set_ylabel('Процент (%)')
    axes[1].set_xticks(x)
    axes[1].set_xticklabels(semester_target_pct.index)
    axes[1].legend()
    axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Анализ оценок
print("=" * 60)
print("АНАЛИЗ ОЦЕНОК")
print("=" * 60)

# Обработка оценок
def parse_mark(mark):
    if pd.isna(mark):
        return np.nan
    mark_str = str(mark).strip().lower()
    if mark_str in ['5', '4', '3', '2']:
        return int(mark_str)
    elif mark_str in ['з', 'зач']:
        return 5
    elif mark_str in ['ня', 'неявка']:
        return 0
    else:
        return np.nan

data_merged['mark_numeric'] = data_merged['MARK'].apply(parse_mark)
data_merged['balls'] = pd.to_numeric(data_merged['BALLS'], errors='coerce')

# Буквенные оценки
grade_mapping = {
    'A': 5, 'A-': 4.7, 'A+': 5,
    'B': 4, 'B-': 3.7, 'B+': 4.3,
    'C': 3, 'C-': 2.7, 'C+': 3.3,
    'D': 2, 'D-': 1.7, 'D+': 2.3,
    'F': 1
}
data_merged['grade_numeric'] = data_merged['GRADE'].map(grade_mapping)

# Финальная оценка
data_merged['final_score'] = data_merged['balls'].fillna(
    data_merged['grade_numeric']).fillna(data_merged['mark_numeric'])

print(f"\nРаспределение финальных оценок:")
print(data_merged['final_score'].describe())

print(f"\nУникальные значения оценок: {sorted(data_merged['final_score'].dropna().unique())}")

# Визуализация распределения оценок
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Гистограмма всех оценок
axes[0, 0].hist(data_merged['final_score'].dropna(), bins=30, color='steelblue', alpha=0.7, edgecolor='black')
axes[0, 0].set_title('Распределение всех оценок', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Оценка')
axes[0, 0].set_ylabel('Частота')
axes[0, 0].grid(alpha=0.3)

# Boxplot по целевой переменной
if 'target' in data_merged.columns:
    data_with_target = data_merged[data_merged['target'].notna()]
    bp_data = [data_with_target[data_with_target['target']==0]['final_score'].dropna(),
               data_with_target[data_with_target['target']==1]['final_score'].dropna()]
    axes[0, 1].boxplot(bp_data, labels=['Не выпустился', 'Выпустился'])
    axes[0, 1].set_title('Распределение оценок по целевой переменной', fontsize=12, fontweight='bold')
    axes[0, 1].set_ylabel('Оценка')
    axes[0, 1].grid(axis='y', alpha=0.3)

# Средние оценки по семестрам
semester_avg = data_merged.groupby('SEMESTER')['final_score'].mean()
axes[1, 0].plot(semester_avg.index, semester_avg.values, marker='o', linewidth=2, markersize=8, color='coral')
axes[1, 0].set_title('Средняя оценка по семестрам', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Семестр')
axes[1, 0].set_ylabel('Средняя оценка')
axes[1, 0].grid(alpha=0.3)

# Типы оценок
if 'TYPE' in data_merged.columns:
    type_dist = data_merged['TYPE'].value_counts()
    axes[1, 1].barh(range(len(type_dist)), type_dist.values, color='teal', alpha=0.7)
    axes[1, 1].set_yticks(range(len(type_dist)))
    axes[1, 1].set_yticklabels(type_dist.index)
    axes[1, 1].set_title('Распределение по типам оценок', fontsize=12, fontweight='bold')
    axes[1, 1].set_xlabel('Количество записей')
    axes[1, 1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Анализ по факультетам и направлениям
if 'Факультет' in data_merged.columns and 'target' in data_merged.columns:
    print("=" * 60)
    print("АНАЛИЗ ПО ФАКУЛЬТЕТАМ И НАПРАВЛЕНИЯМ")
    print("=" * 60)
    
    # Уникальные студенты с целевой переменной
    students_df = data_merged[['PK', 'Факультет', 'Направление', 'target']].drop_duplicates(subset=['PK'])
    
    # Анализ по факультетам
    faculty_target = students_df.groupby(['Факультет', 'target']).size().unstack(fill_value=0)
    faculty_target_pct = faculty_target.div(faculty_target.sum(axis=1), axis=0) * 100
    
    print("\nДоля выпустившихся по факультетам:")
    print(faculty_target_pct.sort_values(1, ascending=False))
    
    # Визуализация
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Факультеты
    top_faculties = faculty_target_pct.sort_values(1, ascending=False).head(10)
    x_pos = np.arange(len(top_faculties))
    
    axes[0].barh(x_pos, top_faculties[1], color='#2ecc71', alpha=0.7, label='Выпустился')
    axes[0].barh(x_pos, top_faculties[0], left=top_faculties[1], color='#e74c3c', alpha=0.7, label='Не выпустился')
    axes[0].set_yticks(x_pos)
    axes[0].set_yticklabels(top_faculties.index, fontsize=9)
    axes[0].set_xlabel('Процент студентов (%)')
    axes[0].set_title('Топ-10 факультетов по доле выпустившихся', fontsize=12, fontweight='bold')
    axes[0].legend()
    axes[0].grid(axis='x', alpha=0.3)
    
    # Направления
    if 'Направление' in students_df.columns:
        direction_target = students_df.groupby(['Направление', 'target']).size().unstack(fill_value=0)
        direction_target_pct = direction_target.div(direction_target.sum(axis=1), axis=0) * 100
        top_directions = direction_target_pct.sort_values(1, ascending=False).head(10)
        
        x_pos2 = np.arange(len(top_directions))
        axes[1].barh(x_pos2, top_directions[1], color='#3498db', alpha=0.7, label='Выпустился')
        axes[1].barh(x_pos2, top_directions[0], left=top_directions[1], color='#e74c3c', alpha=0.7, label='Не выпустился')
        axes[1].set_yticks(x_pos2)
        axes[1].set_yticklabels(top_directions.index, fontsize=9)
        axes[1].set_xlabel('Процент студентов (%)')
        axes[1].set_title('Топ-10 направлений по доле выпустившихся', fontsize=12, fontweight='bold')
        axes[1].legend()
        axes[1].grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.show()


## 4. Корреляционный анализ


In [ ]:
# Создание признаков на уровне студента для корреляционного анализа
print("=" * 60)
print("СОЗДАНИЕ ПРИЗНАКОВ ДЛЯ КОРРЕЛЯЦИОННОГО АНАЛИЗА")
print("=" * 60)

# Фильтруем данные только для первых 4 семестров (первые 2 курса)
data_train = data_merged[data_merged['SEMESTER'] <= 4].copy()

# Создаем признаки на уровне студента
student_features = []
for student_id, student_data in data_train.groupby('PK'):
    features = {'PK': student_id}
    
    # Статистики по оценкам
    scores = student_data['final_score'].dropna()
    if len(scores) > 0:
        features['avg_score'] = scores.mean()
        features['median_score'] = scores.median()
        features['min_score'] = scores.min()
        features['max_score'] = scores.max()
        features['std_score'] = scores.std() if len(scores) > 1 else 0
        features['score_count'] = len(scores)
        features['success_rate'] = (scores >= 3).sum() / len(scores)
        features['excellent_rate'] = (scores >= 4.5).sum() / len(scores)
    else:
        features.update({
            'avg_score': 0, 'median_score': 0, 'min_score': 0,
            'max_score': 0, 'std_score': 0, 'score_count': 0,
            'success_rate': 0, 'excellent_rate': 0
        })
    
    # Статистики по семестрам
    semesters = sorted(student_data['SEMESTER'].unique())
    features['semester_count'] = len(semesters)
    
    # Динамика оценок
    if len(semesters) >= 2:
        sem_scores = []
        for sem in semesters:
            sem_data = student_data[student_data['SEMESTER'] == sem]
            if len(sem_data) > 0:
                sem_scores.append(sem_data['final_score'].dropna().mean())
        if len(sem_scores) >= 2:
            features['score_trend'] = sem_scores[-1] - sem_scores[0]
        else:
            features['score_trend'] = 0
    else:
        features['score_trend'] = 0
    
    # Типы оценок
    if 'TYPE' in student_data.columns:
        features['exams_count'] = (student_data['TYPE'] == 'экз').sum()
        features['credits_count'] = (student_data['TYPE'] == 'зач').sum()
    
    # Информация о студенте
    if 'target' in student_data.columns:
        features['target'] = student_data['target'].iloc[0]
    if 'год поступления' in student_data.columns:
        features['admission_year'] = student_data['год поступления'].iloc[0]
    
    student_features.append(features)

features_df = pd.DataFrame(student_features)
features_df = features_df.fillna(0)

print(f"✓ Создано признаков для {len(features_df):,} студентов")
print(f"  Количество признаков: {len(features_df.columns) - 1}")


In [ ]:
# Корреляционный анализ
if 'target' in features_df.columns:
    # Выбираем числовые признаки
    numeric_cols = features_df.select_dtypes(include=[np.number]).columns.tolist()
    if 'target' in numeric_cols:
        numeric_cols.remove('target')
    
    # Вычисляем корреляции с целевой переменной
    correlations = features_df[numeric_cols + ['target']].corr()['target'].sort_values(ascending=False)
    
    print("=" * 60)
    print("КОРРЕЛЯЦИИ С ЦЕЛЕВОЙ ПЕРЕМЕННОЙ")
    print("=" * 60)
    print("\nТоп-10 признаков с наибольшей корреляцией:")
    print(correlations[1:11])
    
    print("\nТоп-10 признаков с наименьшей корреляцией:")
    print(correlations[-10:])
    
    # Визуализация корреляций
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Барплот корреляций с целевой переменной
    top_corr = correlations[1:11].sort_values()
    axes[0].barh(range(len(top_corr)), top_corr.values, color='steelblue', alpha=0.7)
    axes[0].set_yticks(range(len(top_corr)))
    axes[0].set_yticklabels(top_corr.index, fontsize=9)
    axes[0].set_xlabel('Корреляция с целевой переменной')
    axes[0].set_title('Топ-10 признаков по корреляции с целевой переменной', fontsize=12, fontweight='bold')
    axes[0].axvline(x=0, color='red', linestyle='--', alpha=0.5)
    axes[0].grid(axis='x', alpha=0.3)
    
    # Матрица корреляций для топ признаков
    top_features = correlations[1:11].index.tolist()
    corr_matrix = features_df[top_features + ['target']].corr()
    
    sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
                square=True, linewidths=1, cbar_kws={"shrink": 0.8}, ax=axes[1])
    axes[1].set_title('Матрица корреляций топ-10 признаков', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.show()


## 5. Тестирование гипотез

### Гипотезы для проверки:
1. **Гипотеза 1**: Студенты с более высоким средним баллом чаще выпускаются
2. **Гипотеза 2**: Студенты с положительной динамикой оценок (улучшение по семестрам) чаще выпускаются
3. **Гипотеза 3**: Процент успешных оценок (>= 3) положительно коррелирует с выпуском
4. **Гипотеза 4**: Количество семестров с данными влияет на вероятность выпуска
5. **Гипотеза 5**: Стабильность оценок (низкое стандартное отклонение) связана с выпуском


In [ ]:
# Гипотеза 1: Студенты с более высоким средним баллом чаще выпускаются
if 'target' in features_df.columns and 'avg_score' in features_df.columns:
    print("=" * 60)
    print("ГИПОТЕЗА 1: Студенты с более высоким средним баллом чаще выпускаются")
    print("=" * 60)
    
    # Разделяем на группы
    graduated = features_df[features_df['target'] == 1]['avg_score']
    not_graduated = features_df[features_df['target'] == 0]['avg_score']
    
    print(f"\nСредний балл выпустившихся: {graduated.mean():.2f}")
    print(f"Средний балл не выпустившихся: {not_graduated.mean():.2f}")
    print(f"Разница: {graduated.mean() - not_graduated.mean():.2f}")
    
    # Статистический тест
    from scipy import stats
    t_stat, p_value = stats.ttest_ind(graduated, not_graduated)
    print(f"\nT-тест:")
    print(f"  T-статистика: {t_stat:.4f}")
    print(f"  P-value: {p_value:.4e}")
    print(f"  Гипотеза {'подтверждается' if p_value < 0.05 else 'НЕ подтверждается'} (p < 0.05)")
    
    # Визуализация
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Распределение средних баллов
    axes[0].hist(not_graduated, bins=30, alpha=0.6, label='Не выпустился', color='#e74c3c', edgecolor='black')
    axes[0].hist(graduated, bins=30, alpha=0.6, label='Выпустился', color='#2ecc71', edgecolor='black')
    axes[0].set_xlabel('Средний балл')
    axes[0].set_ylabel('Количество студентов')
    axes[0].set_title('Распределение средних баллов', fontsize=12, fontweight='bold')
    axes[0].legend()
    axes[0].grid(alpha=0.3)
    
    # Boxplot
    bp_data = [not_graduated, graduated]
    axes[1].boxplot(bp_data, labels=['Не выпустился', 'Выпустился'])
    axes[1].set_ylabel('Средний балл')
    axes[1].set_title('Сравнение средних баллов', fontsize=12, fontweight='bold')
    axes[1].grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n✓ ВЫВОД: {'Студенты с более высоким средним баллом действительно чаще выпускаются' if p_value < 0.05 else 'Нет статистически значимой разницы в средних баллах'}")


In [ ]:
# Гипотеза 2: Студенты с положительной динамикой оценок чаще выпускаются
if 'target' in features_df.columns and 'score_trend' in features_df.columns:
    print("=" * 60)
    print("ГИПОТЕЗА 2: Студенты с положительной динамикой оценок чаще выпускаются")
    print("=" * 60)
    
    # Разделяем на группы
    graduated = features_df[features_df['target'] == 1]['score_trend']
    not_graduated = features_df[features_df['target'] == 0]['score_trend']
    
    print(f"\nСредняя динамика выпустившихся: {graduated.mean():.3f}")
    print(f"Средняя динамика не выпустившихся: {not_graduated.mean():.3f}")
    print(f"Разница: {graduated.mean() - not_graduated.mean():.3f}")
    
    # Процент студентов с положительной динамикой
    pos_trend_graduated = (graduated > 0).sum() / len(graduated) * 100
    pos_trend_not_graduated = (not_graduated > 0).sum() / len(not_graduated) * 100
    
    print(f"\nПроцент с положительной динамикой:")
    print(f"  Выпустившиеся: {pos_trend_graduated:.1f}%")
    print(f"  Не выпустившиеся: {pos_trend_not_graduated:.1f}%")
    
    # Статистический тест
    t_stat, p_value = stats.ttest_ind(graduated, not_graduated)
    print(f"\nT-тест:")
    print(f"  T-статистика: {t_stat:.4f}")
    print(f"  P-value: {p_value:.4e}")
    print(f"  Гипотеза {'подтверждается' if p_value < 0.05 else 'НЕ подтверждается'} (p < 0.05)")
    
    # Визуализация
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Распределение динамики
    axes[0].hist(not_graduated, bins=30, alpha=0.6, label='Не выпустился', color='#e74c3c', edgecolor='black')
    axes[0].hist(graduated, bins=30, alpha=0.6, label='Выпустился', color='#2ecc71', edgecolor='black')
    axes[0].axvline(x=0, color='black', linestyle='--', linewidth=2, label='Нет изменений')
    axes[0].set_xlabel('Динамика оценок (разница между последним и первым семестром)')
    axes[0].set_ylabel('Количество студентов')
    axes[0].set_title('Распределение динамики оценок', fontsize=12, fontweight='bold')
    axes[0].legend()
    axes[0].grid(alpha=0.3)
    
    # Boxplot
    bp_data = [not_graduated, graduated]
    axes[1].boxplot(bp_data, labels=['Не выпустился', 'Выпустился'])
    axes[1].axhline(y=0, color='red', linestyle='--', linewidth=2)
    axes[1].set_ylabel('Динамика оценок')
    axes[1].set_title('Сравнение динамики оценок', fontsize=12, fontweight='bold')
    axes[1].grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n✓ ВЫВОД: {'Студенты с положительной динамикой оценок чаще выпускаются' if p_value < 0.05 else 'Нет статистически значимой связи между динамикой и выпуском'}")


In [ ]:
# Гипотеза 3: Процент успешных оценок (>= 3) положительно коррелирует с выпуском
if 'target' in features_df.columns and 'success_rate' in features_df.columns:
    print("=" * 60)
    print("ГИПОТЕЗА 3: Процент успешных оценок положительно коррелирует с выпуском")
    print("=" * 60)
    
    # Разделяем на группы
    graduated = features_df[features_df['target'] == 1]['success_rate']
    not_graduated = features_df[features_df['target'] == 0]['success_rate']
    
    print(f"\nСредний процент успешных оценок выпустившихся: {graduated.mean()*100:.1f}%")
    print(f"Средний процент успешных оценок не выпустившихся: {not_graduated.mean()*100:.1f}%")
    print(f"Разница: {(graduated.mean() - not_graduated.mean())*100:.1f}%")
    
    # Корреляция
    corr = features_df[['success_rate', 'target']].corr().iloc[0, 1]
    print(f"\nКорреляция между процентом успешных оценок и выпуском: {corr:.4f}")
    
    # Статистический тест
    t_stat, p_value = stats.ttest_ind(graduated, not_graduated)
    print(f"\nT-тест:")
    print(f"  T-статистика: {t_stat:.4f}")
    print(f"  P-value: {p_value:.4e}")
    print(f"  Гипотеза {'подтверждается' if p_value < 0.05 else 'НЕ подтверждается'} (p < 0.05)")
    
    # Визуализация
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Распределение процента успешных оценок
    axes[0].hist(not_graduated * 100, bins=30, alpha=0.6, label='Не выпустился', color='#e74c3c', edgecolor='black')
    axes[0].hist(graduated * 100, bins=30, alpha=0.6, label='Выпустился', color='#2ecc71', edgecolor='black')
    axes[0].set_xlabel('Процент успешных оценок (%)')
    axes[0].set_ylabel('Количество студентов')
    axes[0].set_title('Распределение процента успешных оценок', fontsize=12, fontweight='bold')
    axes[0].legend()
    axes[0].grid(alpha=0.3)
    
    # Scatter plot с регрессией
    sample_size = min(1000, len(features_df))
    sample_df = features_df.sample(sample_size)
    axes[1].scatter(sample_df['success_rate'] * 100, sample_df['target'], alpha=0.3, s=20)
    z = np.polyfit(sample_df['success_rate'] * 100, sample_df['target'], 1)
    p = np.poly1d(z)
    axes[1].plot(sample_df['success_rate'] * 100, p(sample_df['success_rate'] * 100), "r--", linewidth=2)
    axes[1].set_xlabel('Процент успешных оценок (%)')
    axes[1].set_ylabel('Выпуск (0/1)')
    axes[1].set_title(f'Зависимость выпуска от процента успешных оценок\n(корреляция: {corr:.3f})', fontsize=12, fontweight='bold')
    axes[1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n✓ ВЫВОД: {'Процент успешных оценок положительно коррелирует с выпуском' if p_value < 0.05 else 'Нет статистически значимой связи'}")


In [ ]:
# Гипотеза 4: Количество семестров с данными влияет на вероятность выпуска
if 'target' in features_df.columns and 'semester_count' in features_df.columns:
    print("=" * 60)
    print("ГИПОТЕЗА 4: Количество семестров с данными влияет на вероятность выпуска")
    print("=" * 60)
    
    # Анализ по количеству семестров
    semester_analysis = features_df.groupby(['semester_count', 'target']).size().unstack(fill_value=0)
    semester_analysis_pct = semester_analysis.div(semester_analysis.sum(axis=1), axis=0) * 100
    
    print("\nДоля выпустившихся по количеству семестров с данными:")
    print(semester_analysis_pct)
    
    # Статистический тест (Chi-square)
    from scipy.stats import chi2_contingency
    chi2, p_value, dof, expected = chi2_contingency(semester_analysis)
    print(f"\nChi-square тест:")
    print(f"  Chi-square статистика: {chi2:.4f}")
    print(f"  P-value: {p_value:.4e}")
    print(f"  Гипотеза {'подтверждается' if p_value < 0.05 else 'НЕ подтверждается'} (p < 0.05)")
    
    # Визуализация
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Столбчатая диаграмма
    x_pos = np.arange(len(semester_analysis_pct))
    width = 0.35
    axes[0].bar(x_pos - width/2, semester_analysis_pct[0], width, label='Не выпустился', alpha=0.7, color='#e74c3c')
    axes[0].bar(x_pos + width/2, semester_analysis_pct[1], width, label='Выпустился', alpha=0.7, color='#2ecc71')
    axes[0].set_xlabel('Количество семестров с данными')
    axes[0].set_ylabel('Процент студентов (%)')
    axes[0].set_title('Доля выпустившихся по количеству семестров', fontsize=12, fontweight='bold')
    axes[0].set_xticks(x_pos)
    axes[0].set_xticklabels(semester_analysis_pct.index)
    axes[0].legend()
    axes[0].grid(axis='y', alpha=0.3)
    
    # Линейный график доли выпустившихся
    axes[1].plot(semester_analysis_pct.index, semester_analysis_pct[1], marker='o', linewidth=2, markersize=8, color='#2ecc71')
    axes[1].set_xlabel('Количество семестров с данными')
    axes[1].set_ylabel('Доля выпустившихся (%)')
    axes[1].set_title('Зависимость выпуска от количества семестров', fontsize=12, fontweight='bold')
    axes[1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n✓ ВЫВОД: {'Количество семестров с данными статистически значимо влияет на выпуск' if p_value < 0.05 else 'Нет статистически значимой связи'}")


In [ ]:
# Гипотеза 5: Стабильность оценок (низкое стандартное отклонение) связана с выпуском
if 'target' in features_df.columns and 'std_score' in features_df.columns:
    print("=" * 60)
    print("ГИПОТЕЗА 5: Стабильность оценок связана с выпуском")
    print("=" * 60)
    
    # Фильтруем студентов с достаточным количеством оценок для расчета std
    features_with_std = features_df[features_df['score_count'] >= 3].copy()
    
    if len(features_with_std) > 0:
        graduated = features_with_std[features_with_std['target'] == 1]['std_score']
        not_graduated = features_with_std[features_with_std['target'] == 0]['std_score']
        
        print(f"\nСреднее стандартное отклонение выпустившихся: {graduated.mean():.3f}")
        print(f"Среднее стандартное отклонение не выпустившихся: {not_graduated.mean():.3f}")
        print(f"Разница: {graduated.mean() - not_graduated.mean():.3f}")
        
        # Корреляция
        corr = features_with_std[['std_score', 'target']].corr().iloc[0, 1]
        print(f"\nКорреляция между стабильностью и выпуском: {corr:.4f}")
        
        # Статистический тест
        t_stat, p_value = stats.ttest_ind(graduated, not_graduated)
        print(f"\nT-тест:")
        print(f"  T-статистика: {t_stat:.4f}")
        print(f"  P-value: {p_value:.4e}")
        print(f"  Гипотеза {'подтверждается' if p_value < 0.05 else 'НЕ подтверждается'} (p < 0.05)")
        
        # Визуализация
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # Распределение стандартных отклонений
        axes[0].hist(not_graduated, bins=30, alpha=0.6, label='Не выпустился', color='#e74c3c', edgecolor='black')
        axes[0].hist(graduated, bins=30, alpha=0.6, label='Выпустился', color='#2ecc71', edgecolor='black')
        axes[0].set_xlabel('Стандартное отклонение оценок')
        axes[0].set_ylabel('Количество студентов')
        axes[0].set_title('Распределение стабильности оценок\n(ниже = стабильнее)', fontsize=12, fontweight='bold')
        axes[0].legend()
        axes[0].grid(alpha=0.3)
        
        # Boxplot
        bp_data = [not_graduated, graduated]
        axes[1].boxplot(bp_data, labels=['Не выпустился', 'Выпустился'])
        axes[1].set_ylabel('Стандартное отклонение оценок')
        axes[1].set_title('Сравнение стабильности оценок', fontsize=12, fontweight='bold')
        axes[1].grid(axis='y', alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        print(f"\n✓ ВЫВОД: {'Стабильность оценок статистически значимо связана с выпуском' if p_value < 0.05 else 'Нет статистически значимой связи между стабильностью и выпуском'}")
    else:
        print("\nНедостаточно данных для анализа (нужно минимум 3 оценки на студента)")


## 6. Дополнительный анализ: Взаимосвязи между признаками


In [ ]:
# Анализ взаимосвязей между ключевыми признаками
if 'target' in features_df.columns:
    # Выбираем ключевые признаки
    key_features = ['avg_score', 'success_rate', 'excellent_rate', 'score_trend', 
                   'semester_count', 'std_score', 'score_count']
    key_features = [f for f in key_features if f in features_df.columns]
    
    if len(key_features) > 0:
        print("=" * 60)
        print("АНАЛИЗ ВЗАИМОСВЯЗЕЙ МЕЖДУ ПРИЗНАКАМИ")
        print("=" * 60)
        
        # Полная матрица корреляций
        corr_matrix = features_df[key_features + ['target']].corr()
        
        # Визуализация
        plt.figure(figsize=(12, 10))
        sns.heatmap(corr_matrix, annot=True, fmt='.3f', cmap='coolwarm', center=0,
                   square=True, linewidths=1, cbar_kws={"shrink": 0.8}, 
                   vmin=-1, vmax=1, annot_kws={'size': 9})
        plt.title('Матрица корреляций ключевых признаков', fontsize=14, fontweight='bold', pad=20)
        plt.tight_layout()
        plt.show()
        
        # Анализ самых сильных корреляций
        print("\nТоп-5 самых сильных положительных корреляций:")
        corr_pairs = []
        for i in range(len(corr_matrix.columns)):
            for j in range(i+1, len(corr_matrix.columns)):
                corr_pairs.append((corr_matrix.columns[i], corr_matrix.columns[j], 
                                 corr_matrix.iloc[i, j]))
        corr_pairs_sorted = sorted(corr_pairs, key=lambda x: abs(x[2]), reverse=True)
        for feat1, feat2, corr_val in corr_pairs_sorted[:5]:
            print(f"  {feat1} ↔ {feat2}: {corr_val:.3f}")


## 7. Анализ по семестрам отдельно


In [ ]:
# Детальный анализ по каждому семестру
print("=" * 60)
print("АНАЛИЗ ПО СЕМЕСТРАМ")
print("=" * 60)

if 'target' in data_train.columns:
    # Средние оценки по семестрам для разных групп
    semester_analysis = []
    for sem in sorted(data_train['SEMESTER'].unique()):
        sem_data = data_train[data_train['SEMESTER'] == sem]
        
        if 'target' in sem_data.columns:
            graduated_scores = sem_data[sem_data['target'] == 1]['final_score'].dropna()
            not_graduated_scores = sem_data[sem_data['target'] == 0]['final_score'].dropna()
            
            semester_analysis.append({
                'semester': sem,
                'avg_all': sem_data['final_score'].mean(),
                'avg_graduated': graduated_scores.mean() if len(graduated_scores) > 0 else 0,
                'avg_not_graduated': not_graduated_scores.mean() if len(not_graduated_scores) > 0 else 0,
                'count_graduated': len(graduated_scores),
                'count_not_graduated': len(not_graduated_scores)
            })
    
    semester_df = pd.DataFrame(semester_analysis)
    print("\nСредние оценки по семестрам:")
    print(semester_df)
    
    # Визуализация
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Линейный график средних оценок
    axes[0].plot(semester_df['semester'], semester_df['avg_graduated'], 
                marker='o', linewidth=2, markersize=8, label='Выпустился', color='#2ecc71')
    axes[0].plot(semester_df['semester'], semester_df['avg_not_graduated'], 
                marker='s', linewidth=2, markersize=8, label='Не выпустился', color='#e74c3c')
    axes[0].set_xlabel('Семестр')
    axes[0].set_ylabel('Средняя оценка')
    axes[0].set_title('Средние оценки по семестрам', fontsize=12, fontweight='bold')
    axes[0].legend()
    axes[0].grid(alpha=0.3)
    axes[0].set_xticks(semester_df['semester'])
    
    # Разница между группами
    semester_df['diff'] = semester_df['avg_graduated'] - semester_df['avg_not_graduated']
    axes[1].bar(semester_df['semester'], semester_df['diff'], color='steelblue', alpha=0.7)
    axes[1].set_xlabel('Семестр')
    axes[1].set_ylabel('Разница в средних оценках')
    axes[1].set_title('Разница между средними оценками\n(положительная = выпустившиеся лучше)', 
                     fontsize=12, fontweight='bold')
    axes[1].axhline(y=0, color='red', linestyle='--', linewidth=2)
    axes[1].grid(axis='y', alpha=0.3)
    axes[1].set_xticks(semester_df['semester'])
    
    plt.tight_layout()
    plt.show()


## 8. Выводы и закономерности

### Основные находки:

#### 1. **Связь среднего балла с выпуском**
- Студенты с более высоким средним баллом статистически значимо чаще выпускаются
- Это одна из самых сильных корреляций

#### 2. **Динамика оценок**
- Положительная динамика (улучшение оценок) связана с более высокой вероятностью выпуска
- Важна не только абсолютная оценка, но и тенденция

#### 3. **Процент успешных оценок**
- Процент оценок >= 3 (успешных) является важным предиктором
- Студенты с высокой долей успешных оценок чаще выпускаются

#### 4. **Количество семестров**
- Количество семестров с данными может влиять на вероятность выпуска
- Больше данных = более точная оценка потенциала студента

#### 5. **Стабильность оценок**
- Стабильность оценок (низкое стандартное отклонение) также важна
- Нестабильность может указывать на проблемы

### Рекомендации для моделирования:

1. **Важные признаки**:
   - Средний балл студента
   - Процент успешных оценок
   - Динамика оценок (тренд)
   - Стабильность оценок (std)

2. **Временные особенности**:
   - Оценки по каждому семестру отдельно
   - Разница между семестрами
   - Последние оценки (могут быть более релевантными)

3. **Категориальные признаки**:
   - Факультет и направление показывают различия
   - Год поступления может быть важен

4. **Обработка данных**:
   - Использовать только данные за первые 4 семестра (первые 2 курса)
   - Исключить утечки данных
   - Обработать пропуски корректно


In [ ]:
# Финальная сводка ключевых метрик
print("=" * 80)
print("ФИНАЛЬНАЯ СВОДКА ИССЛЕДОВАНИЯ")
print("=" * 80)

if 'target' in features_df.columns:
    print(f"\n📊 Общая статистика:")
    print(f"  • Всего студентов в анализе: {len(features_df):,}")
    print(f"  • Выпустившихся: {(features_df['target'] == 1).sum():,} ({(features_df['target'] == 1).mean()*100:.1f}%)")
    print(f"  • Не выпустившихся: {(features_df['target'] == 0).sum():,} ({(features_df['target'] == 0).mean()*100:.1f}%)")
    
    print(f"\n📈 Ключевые различия между группами:")
    if 'avg_score' in features_df.columns:
        avg_grad = features_df[features_df['target'] == 1]['avg_score'].mean()
        avg_not_grad = features_df[features_df['target'] == 0]['avg_score'].mean()
        print(f"  • Средний балл:")
        print(f"    - Выпустившиеся: {avg_grad:.2f}")
        print(f"    - Не выпустившиеся: {avg_not_grad:.2f}")
        print(f"    - Разница: {avg_grad - avg_not_grad:.2f}")
    
    if 'success_rate' in features_df.columns:
        success_grad = features_df[features_df['target'] == 1]['success_rate'].mean()
        success_not_grad = features_df[features_df['target'] == 0]['success_rate'].mean()
        print(f"  • Процент успешных оценок:")
        print(f"    - Выпустившиеся: {success_grad*100:.1f}%")
        print(f"    - Не выпустившиеся: {success_not_grad*100:.1f}%")
        print(f"    - Разница: {(success_grad - success_not_grad)*100:.1f}%")
    
    print(f"\n✓ Исследование завершено!")
    print(f"  Все гипотезы протестированы")
    print(f"  Выявлены ключевые закономерности")
    print(f"  Данные готовы для построения модели")
